# Load Libraries

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# Data science
import math
import scipy.stats as stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.stats.multitest import multipletests as mt
import random

# Plots
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib import gridspec
from matplotlib.colors import to_hex
import colorcet as cc
from matplotlib.lines import Line2D

# Working with dates
from datetime import date,datetime
import dateutil

# Looping  progress
from tqdm.notebook import tqdm

# Reg expressions
import re

# Pretty table printing
import tabulate

import os
import subprocess

# Misc libraries
from IPython.display import display, HTML
#from IPython.core.display import display, HTML

# Set seaborn figure size, font size, and style
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set(font_scale=1.5)
sns.set_style("white")

# Set Pandas options so we can see our entire dataframe
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000
pd.options.display.max_colwidth = None

# Print our versions of this packages, this allows us to make sure
# we have the working versions we need. 
print(f"Pandas version: {pd.__version__}")

# Prep Environment

In [ ]:
# Remove grey side bars
display(HTML("<style>.container { width:90% !important; }</style>"))

os.chdir('../../../../results/')

# Data prep

In [ ]:
all_res = pd.read_excel('../manuscript/latest/supplemental_datasets/supplemental_dataset_2.xlsx',
                    sheet_name = 'Results')

In [ ]:
# Revert our more human-friendly column names back to computer-friendly ones.
human_to_computer_column_dict = {'Disease' : 'disease', 
                                'ICD10' : 'icd', 
                                'Organism' : 'org', 
                                'Antibody' : 'anti', 
                                'Pair is Associated' : 'pair_is_associated', 
                                'Standard Level' : 'std_lev', 
                                'Replication Status' : 'rep_stat', 
                                'UKB adj p' : 'ukb_per_dis_bh_fdr_corr_nom_p', 
                                'TNX adj p' : 'tnx_per_dis_bh_fdr_corr_p', 
                                'UKB OR' : 'ukb_OR', 
                                'TNX OR' : 'tnx_OR', 
                                'UKB CI' : 'ukb_anti_CI', 
                                'TNX CI' : 'tnx_CI', 
                                'UKB nCase' : 'ukb_nCase',
                                'UKB nControl' : 'ukb_nControl',
                                'TNX nCase' : 'tnx_nCase', 
                                'TNX nControl' : 'tnx_nControl'
}

all_res = all_res.rename(columns = human_to_computer_column_dict)

# Revert human friendly organism names to computationally friendly ones
human_to_computer_org_dict = {
                                'BKV': 'bkv',
                                'C. trachomatis': 'c_trach',
                                'CMV': 'cmv',
                                'EBV': 'ebv',
                                'H. pylori': 'h_pylor',
                                'HBV': 'hbv',
                                'HCV': 'hcv',
                                'HHV-6': 'hhv_6',
                                'HHV-7': 'hhv_7',
                                'HIV': 'hiv',
                                'HPV-16': 'hpv_16',
                                'HPV-18': 'hpv_18',
                                'HSV-1': 'hsv_1',
                                'HSV-2': 'hsv_2',
                                'HTLV-1': 'htlv',
                                'JCV': 'jcv',
                                'KSHV': 'kshv',
                                'MCV': 'mcv',
                                'T. gondii': 't_gond',
                                'VZV': 'vzv'
                            }
all_res.loc[:, 'org'] = all_res.loc[:, 'org'].replace(human_to_computer_org_dict)


human_to_computer_anti_dict = {
                                    'K8.1' : 'K8_1', 
                                    'gE / gI' : 'gE_gl', 
                                    'pp 52' : 'pp52', 
                                    'pp 28' : 'pp28', 
                                    '2mgG' : 'IgG', 
                                    'momp A' : 'momp_A', 
                                    'pp150 Nter' : 'pp150_Nter', 
                                    '1gG' : 'IgG', 
                                    'p101 k' : 'p101_k', 
                                    'tarp-D F2' : 'tarp-D_F2', 
                                    'momp D' : 'momp_D', 
                                    'tarp-D F1' : 'tarp-D_F1', 
                                    'VCA p18' : 'VCA_p18'
}
all_res.loc[:, 'anti'] = all_res.loc[:, 'anti'].replace(human_to_computer_anti_dict)

human_to_computer_std_lev_dict = {
    'Unknown' : 'unk',
    'Exp. Negative' : 'exp_neg'
}
all_res.loc[:, 'std_lev'] = all_res.loc[:, 'std_lev'].replace(human_to_computer_std_lev_dict)

human_to_computer_rep_dict = {
    'Did not attempt' : 'did_not_attempt', 
    'Replicated' : 'replicated', 
    'Failed Replication' : 'did_not', 
    'Could not attempt' : 'could_not'
}
all_res.loc[:, 'rep_stat'] = all_res.loc[:, 'rep_stat'].replace(human_to_computer_rep_dict)

In [ ]:
res = all_res.loc[all_res['rep_stat'] == 'replicated', :]

## Get Abbreviated Disease Names

In [ ]:
dis_dat = pd.read_excel('../dicts/dis_abbrev_dict.xlsx')
res = pd.merge(res, dis_dat[['icd', 'dis_abbrev']], 
         left_on = 'icd', 
         right_on = 'icd',
         how = 'left')


res.loc[:, 'dis_abbrev_lab'] = "[" + res.loc[:, 'icd'] + "] " + res.loc[:, 'dis_abbrev']

## Get nice organism names

In [ ]:
vir_dat = pd.read_excel('../dicts/antigen_dict.xlsx')
vir_dat = vir_dat[['Tag', 'Organism', 'Family', 'Species', 'Baltimore']].drop_duplicates()

res = pd.merge(res, vir_dat, 
         left_on = 'org', 
         right_on = 'Tag',
         how = 'left')

In [ ]:
simp_org_name = {
                      'Human Polyomavirus BKV': 'BKV',
                      'Epstein-Barr Virus': 'EBV',
                      'Human Herpesvirus-7': 'HHV-7',
                      'Herpes Simplex virus-1': 'HSV-1',
                      'Herpes Simplex virus-2': 'HSV-2',
                      'Human Herpesvirus-6': 'HHV-6',
                      "Kaposi's Sarcoma-Associated Herpesvirus": 'KSHV',
                      'Human T-Lymphotropic Virus 1': 'HTLV-1',
                      'Human Immunodeficiency Virus': 'HIV',
                      'Varicella Zoster Virus': 'VZV',
                      'Merkel Cell Polyomavirus': 'MCV',
                      'Human Papillomavirus type-18': 'HPV-18',
                      'Hepatitis C Virus': 'HCV',
                      'Human Polyomavirus JCV': 'JCV',
                      'Human Papillomavirus type-16': 'HPV-16',
                      'Human Cytomegalovirus': 'CMV',
                      'Hepatitis B Virus': 'HBV',

            }

# Horizontal barplot of ICD10 risk/protect split

## Collect data

In [ ]:
# Add blocks
all_res['icd_cat'] = all_res.loc[:, 'icd'].str[:1]
all_res['icd_site'] = all_res.loc[:, 'icd'].str[1:].astype(str)


# Create ICD10 Blocks with descriptions
all_res['icd_block'] = np.nan
all_res['icd_descr'] = np.nan



all_res.loc[((all_res['icd_cat'].isin(['A', 'B']))), 'icd_block'] = 'A00-B99'

all_res.loc[(
                (all_res['icd_cat'] == 'C') |
    
                ((all_res['icd_cat'] == 'D') & (all_res['icd_site'].astype(int) < 50))
            ), 'icd_block'] = 'C00-D49'


all_res.loc[((all_res['icd_cat'] == 'D') & (all_res['icd_site'].astype(int) >= 50)), 
            'icd_block'] = 'D50-D89'

all_res.loc[(all_res['icd_cat'] == 'E'), 'icd_block'] = 'E00-E90'

all_res.loc[(all_res['icd_cat'] == 'F'), 'icd_block'] = 'F00-F99'

all_res.loc[(all_res['icd_cat'] == 'G'), 'icd_block'] = 'G00-G99'



all_res.loc[((all_res['icd_cat'] == 'H') & (all_res['icd_site'].astype(int) < 60)), 
            'icd_block'] = 'H00-H59'


all_res.loc[((all_res['icd_cat'] == 'H') & (all_res['icd_site'].astype(int) >= 60)), 
            'icd_block'] = 'H60-H95'

all_res.loc[(all_res['icd_cat'] == 'I'), 'icd_block'] = 'I00-I99'
all_res.loc[(all_res['icd_cat'] == 'J'), 'icd_block'] = 'J00-J99'
all_res.loc[(all_res['icd_cat'] == 'K'), 'icd_block'] = 'K00-K93'
all_res.loc[(all_res['icd_cat'] == 'L'), 'icd_block'] = 'L00-L99'
all_res.loc[(all_res['icd_cat'] == 'M'), 'icd_block'] = 'M00-M99'
all_res.loc[(all_res['icd_cat'] == 'N'), 'icd_block'] = 'N00-N99'
all_res.loc[(all_res['icd_cat'] == 'O'), 'icd_block'] = 'O00-O99'



all_res.loc[((all_res['icd_cat'].isin(['A', 'B']))), 'icd_descr'] = '[A00-B99] Infectious'

all_res.loc[(
                (all_res['icd_cat'] == 'C') |
    
                ((all_res['icd_cat'] == 'D') & (all_res['icd_site'].astype(int) < 50))
            ), 'icd_descr'] = '[C00-D49] Neoplasms'


all_res.loc[((all_res['icd_cat'] == 'D') & (all_res['icd_site'].astype(int) >= 50)), 
            'icd_descr'] = '[D50-D89] Blood'

all_res.loc[(all_res['icd_cat'] == 'E'), 'icd_descr'] = '[E00-E90] Endocrine, Nutritional, Metabolic'

all_res.loc[(all_res['icd_cat'] == 'F'), 'icd_descr'] = '[F00-F99] Mental, Behavioral'

all_res.loc[(all_res['icd_cat'] == 'G'), 'icd_descr'] = '[G00-G99] Nervous'



all_res.loc[((all_res['icd_cat'] == 'H') & (all_res['icd_site'].astype(int) < 60)), 
            'icd_descr'] = '[H00-H59] Eye'


all_res.loc[((all_res['icd_cat'] == 'H') & (all_res['icd_site'].astype(int) >= 60)), 
            'icd_descr'] = '[H60-H95] Ear'

all_res.loc[(all_res['icd_cat'] == 'I'), 'icd_descr'] = '[I00-I99] Circulatory'
all_res.loc[(all_res['icd_cat'] == 'J'), 'icd_descr'] = '[J00-J99] Respiratory'
all_res.loc[(all_res['icd_cat'] == 'K'), 'icd_descr'] = '[K00-K93] Digestive'
all_res.loc[(all_res['icd_cat'] == 'L'), 'icd_descr'] = '[L00-L99] Skin'
all_res.loc[(all_res['icd_cat'] == 'M'), 'icd_descr'] = '[M00-M99] Musculoskeletal'
all_res.loc[(all_res['icd_cat'] == 'N'), 'icd_descr'] = '[N00-N99] Genitourinary'
all_res.loc[(all_res['icd_cat'] == 'O'), 'icd_descr'] = '[O00-O99] Pregnancy, Childbirth'

In [ ]:
# Split up results into risk vs protect
all_res['is_risk'] = False
all_res.loc[all_res['ukb_OR'] > 1, 'is_risk'] = True

dat = all_res.loc[all_res['rep_stat'] == 'replicated', :]

pl_dat = pd.DataFrame(dat.groupby(['org', 'icd_descr', 
                                   'rep_stat' ]).size()).reset_index(drop = False)
pl_dat.columns = ['org', 'icd_descr', 'rep_stat',  'count']


# Push in the nicer org names
pl_dat = pl_dat.merge(vir_dat, left_on = 'org', right_on = 'Tag', how = 'left')
pl_dat.loc[:, 'Baltimore']  = pl_dat.loc[:, 'Baltimore'].fillna(0)
pl_dat.loc[:, 'Baltimore']  = pl_dat.loc[:, 'Baltimore'].astype(int)

pl_dat['simple_name'] = pl_dat.loc[:, 'Organism'].replace(simp_org_name)

In [ ]:
spl_ls = []

for _, curr_row in tqdm(pl_dat.iterrows(), total = len(pl_dat)):

    curr_row_ls = curr_row.tolist()

    curr_org = curr_row['org']
    curr_dis = curr_row['icd_descr']

    curr_block = dat.loc[((dat['org'] == curr_org) & 
                      (dat['icd_descr'] == curr_dis) &
                     (dat['rep_stat'] == 'replicated')), :]

    curr_risk_dict = curr_block['is_risk'].value_counts().to_dict()


    if False in curr_risk_dict.keys():
        curr_protect = curr_risk_dict[False]
    else:
        curr_protect = 0

    if True in curr_risk_dict.keys():
        curr_risk = curr_risk_dict[True]
    else:
        curr_risk = 0

    curr_row_ls.extend([curr_risk, curr_protect])

    spl_ls.append(curr_row_ls)
    
up_pl_dat = pd.DataFrame(spl_ls, columns = pl_dat.columns.tolist() + ['risk_cnt', 'protect_cnt'])


pl_dat = up_pl_dat.copy(deep = True)

viruses = pl_dat.copy(deep = True)
viruses = viruses.loc[((viruses['Tag'] != 'c_trach') & 
                       (viruses['Tag'] != 'h_pylor') &
                       (viruses['Tag'] != 't_gond')), :]

viruses['formatted_name'] = viruses.loc[:, 'simple_name']
viruses['org_type'] = 'virus'

microbes = pl_dat.copy(deep = True)
microbes = microbes.loc[((microbes['Tag'] == 'c_trach') | 
                         (microbes['Tag'] == 'h_pylor') |
                         (microbes['Tag'] == 't_gond')), :]

corr_name_ls = []
for _, curr_row in tqdm(microbes.iterrows(), total = len(microbes)):
    
    curr_name = curr_row['Organism']
    
    if curr_name == 'Toxoplasma gondii':
        corr_name_ls.append('$\it{T. gondii}$')
    
    elif curr_name == 'Helicobacter pylori':
        corr_name_ls.append('$\it{H. pylori}$')

    elif curr_name == 'Chlamydia trachomatis': 
        corr_name_ls.append('$\it{C. trachomatis}$')   
        
microbes['formatted_name'] = corr_name_ls
microbes['org_type'] = 'microbe'

fin_pl = pd.concat([microbes, viruses])

rep = fin_pl.loc[fin_pl['rep_stat'] == 'replicated', :]

## Do some counting!

In [ ]:
tot_df = rep.groupby('org').agg({'count' : 'sum', 'risk_cnt' : 'sum', 'protect_cnt' : 'sum'})
tot_df = tot_df.reset_index(drop = False)

tots = pd.DataFrame(columns = rep.columns)

for curr_row in tot_df.values:

    curr_org = curr_row[0]
    curr_tot_cnt = curr_row[1]
    curr_tot_risk = curr_row[2]
    curr_tot_prot = curr_row[3]

    curr_org_info = rep.loc[rep['org'] == curr_org, :].iloc[0].copy(deep = True)

    curr_org_info['icd_descr'] = '[A00 - O99] All Diseases'
    curr_org_info['count'] = curr_tot_cnt
    curr_org_info['risk_cnt'] = curr_tot_risk
    curr_org_info['protect_cnt'] = curr_tot_prot

    tots = tots.append(curr_org_info, ignore_index = True)
    
tots['neg_protect_cnt'] = tots.loc[:, 'protect_cnt'] * -1

# Ugly way to manuallys specify org order
ordered_orgs = [

                # Bacteria
                ('$\\it{C. trachomatis}$', 0),
                ('$\\it{H. pylori}$', 1),
                
                # Alveolata
                ('$\\it{T. gondii}$', 2),
                
                # Riboviria - HBV more closely related to HIV and HTLV-1 than HCV atleast by phylo
                ('HBV', 3),
                ('HCV', 4),
                ('HIV', 5),
                ('HTLV1', 6),
                
                # Papovaviricetes
                ('BKV', 7),
                ('JCV', 8),
                ('MCV', 9),
                ('HPV16', 10),
                ('HPV18', 11),
                
                # Herpes
                ('CMV', 12),
                ('EBV', 13),
                ('HSV1', 14),
                ('HSV2', 15),
                ('HHV6', 16),
                ('HHV7', 17),
                ('VZV', 18),
                ('KSHV/HHV8', 19),
            
                ]

corr_x_names = [x[0] for x in ordered_orgs]
ordered_orgs_df = pd.DataFrame(ordered_orgs)
ordered_orgs_df.columns = ['formatted_name', 'enum_num']

tots.loc[:, 'formatted_name'] = tots.loc[:, 'formatted_name'].str.replace('-', '')
tots = tots.merge(ordered_orgs_df, on = 'formatted_name', how = 'left')

# Per Leah, bold the ICD description but not the code range
rep.loc[:, 'icd_descr'] = rep.loc[:, 'icd_descr'].str.replace('] ', '] $\\bf{', regex = False) + '}$'
tots.loc[:, 'icd_descr'] = tots.loc[:, 'icd_descr'].str.replace('] ', '] $\\bf{', regex = False) + '}$'


## Little more processing

In [ ]:
tots = tots.sort_values('enum_num', ascending = False)
clean = tots.loc[:, ['org', 'formatted_name', 'risk_cnt', 'protect_cnt']]
  

clean.loc[:, 'protect_cnt'] = -1 * clean.loc[:, 'protect_cnt']
clean['tot'] = clean['risk_cnt'] + (clean['protect_cnt'] * -1)

# per Matt for v04
clean = clean.sort_values('tot')

## Generate plot

In [ ]:
from matplotlib.patches import  FancyArrowPatch as Arrow
CON_COLOR =  '#4A6990'
CASE_COLOR = '#bb3d2e'

BAR_LAB_FONT_SIZE = 16

fig, ax = plt.subplots(figsize = (6, 15), facecolor = 'white')

p1 = ax.barh(y = clean['formatted_name'], width = clean['risk_cnt'], left = 0, 
        align = 'center', color = CASE_COLOR)
p2 = ax.barh(y = clean['formatted_name'], width = clean['protect_cnt'], left = 0, 
        align = 'center', color = CON_COLOR)

ax.axvline(0, color = '#242424', linewidth = 1)

risk_labs = ax.bar_label(p1, label_type='edge', color = '#242424', fontsize = BAR_LAB_FONT_SIZE)
prot_labs = ax.bar_label(p2, label_type='edge', color = '#242424', fontsize = BAR_LAB_FONT_SIZE)

ax.set_xlim(-25, 25)

fin_labs = []
for curr_text in ax.get_xticklabels():
    #print(curr_text)
    curr_text_val = curr_text.get_text()
    #print(curr_text_val)
    if u"\u2212" in curr_text_val:
        curr_text_val = re.sub(u"\u2212", "", curr_text_val)
        #print(curr_text_val)
        curr_text.set_text(curr_text_val)
    fin_labs.append(curr_text)
        
        
ax.set_xticklabels(fin_labs)

for curr_text in risk_labs:
    if(int(curr_text.get_text())) == 0:
        curr_text.set_x(2)
        print(curr_text)
        continue


for curr_text in prot_labs:
    if(int(curr_text.get_text())) == 0:
        curr_text.set_x(-10)
        print(curr_text)
        continue
        
    curr_text.set_text(str(int(curr_text.get_text()) * -1)) 
   
ARROW_Y = -0.05
TEXT_Y  = -0.075
X_LAB_Y = -0.125
ARROW_TEXT_FONT = {
            'family': 'DejaVu Sans',
            'color':  'black',
            'weight': 'normal',
            'size': 13,  
}

X_LAB_FONT = {
            'family': 'DejaVu Sans',
            'color':  'black',
            'weight': 'normal',
            'size': 15,  
}

ARROW_LEN = 0.375
ARROW_OFF_CENTER = 0.02
CENTER = 0.5


risk_ar_st_x = CENTER + ARROW_OFF_CENTER
risk_ar_end_x = risk_ar_st_x + ARROW_LEN
risk_mid_x = CENTER + (ARROW_LEN / 2)

prot_ar_st_x = CENTER - ARROW_OFF_CENTER
prot_ar_end_x = prot_ar_st_x - ARROW_LEN
prot_mid_x = CENTER - (ARROW_LEN / 2)


risk_ar = Arrow(posA = (risk_ar_st_x, ARROW_Y), posB = (risk_ar_end_x, ARROW_Y), arrowstyle = "-|>",
                color = CASE_COLOR, linewidth = 2, mutation_scale = 25, 
                transform = ax.transAxes)
 
prot_ar = Arrow(posA = (prot_ar_st_x, ARROW_Y), posB = (prot_ar_end_x, ARROW_Y), arrowstyle = "-|>",
                color = CON_COLOR, linewidth = 2, mutation_scale = 25, 
                transform = ax.transAxes)    

fig.patches.extend([risk_ar, prot_ar])

ax.text(x = risk_mid_x, 
        y = TEXT_Y, 
        s = 'Increases Risk\n[OR > 1]',
        ha = 'center', va = 'center',
        transform = ax.transAxes, 
        fontdict = ARROW_TEXT_FONT)


ax.text(x = prot_mid_x, 
        y = TEXT_Y, 
        s = 'Decreases Risk\n[OR < 1]',
        ha = 'center', va = 'center',
        transform = ax.transAxes, 
        fontdict = ARROW_TEXT_FONT)


ax.text(x = 0.5, 
        y = X_LAB_Y, 
        s = 'Number of Replicated Results',
        ha = 'center', va = 'center',
        transform = ax.transAxes, 
        fontdict = X_LAB_FONT)

## Save plot

In [ ]:
out_dir = '../manuscript/figures/supp_fig_3'
fn = f"{out_dir}/supp_fig_3.pdf"
fig.savefig(fn, format = 'pdf', dpi = 600, bbox_inches="tight")